In [1]:
#import image datagenerator library
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.1,
                                   zoom_range=0.1,
                                   horizontal_flip=True)

In [3]:
val_datagen = ImageDataGenerator(rescale = 1./255)

In [4]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_a477f1f4c97f4e2ebc3add26a34b01ee = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='Iw6tRzCrJcew0w-D6rye8_WZ3yYKEO_8ERIjxFxOHedJ',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

streaming_body_1 = client_a477f1f4c97f4e2ebc3add26a34b01ee.get_object(Bucket='lungcancerprediction-donotdelete-pr-lxbjbk81a7xsro', Key='Dataset1.zip')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [5]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [6]:
pwd

'/home/wsuser/work'

In [7]:
import os

In [8]:
filename=os.listdir('/home/wsuser/work/Dataset')

In [9]:
train_transform = train_datagen.flow_from_directory(r'/home/wsuser/work/Dataset/traindata',
                                            target_size=(128,128),
                                            batch_size=64,
                                            class_mode='categorical')

Found 102 images belonging to 2 classes.


In [10]:
test_transform = val_datagen.flow_from_directory(r'/home/wsuser/work/Dataset/testdata',
                                            target_size=(128,128),
                                            batch_size=64,
                                            class_mode='categorical')

Found 39 images belonging to 2 classes.


In [11]:
#to define linear initializations import Sequential
from tensorflow.keras.models import Sequential
#To add layers import Dense
from tensorflow.keras.layers import Dense
# to create a convolution kernel import Convolution2D
from tensorflow.keras.layers import Convolution2D
# Adding Max pooling Layer
from tensorflow.keras.layers import MaxPooling2D
# Adding Flatten Layer
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import Adam

In [12]:
# Initializing the model
model=Sequential()

In [13]:
model.add(Convolution2D(64,(3,3),input_shape=(128,128,3),activation='relu'))
model.add(MaxPooling2D(2,2))

In [14]:
model.add(Flatten()) 

In [15]:
model.output_shape

(None, 254016)

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 254016)            0         
Total params: 1,792
Trainable params: 1,792
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Adding 1st hidden layer
model.add(Dense(kernel_initializer='uniform',activation='relu',units=150))

In [18]:
model.add(Dense(kernel_initializer='uniform',activation='relu',units=68))

In [19]:
model.add(Dense(kernel_initializer='uniform',activation='softmax',units=2))

In [20]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 254016)            0         
_________________________________________________________________
dense (Dense)                (None, 150)               38102550  
_________________________________________________________________
dense_1 (Dense)              (None, 68)                10268     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 138       
Total params: 38,114,748
Trainable params: 38,114,748
Non-trainable params: 0
____________________________________________

In [22]:
res = model.fit(train_transform,steps_per_epoch=102//64,validation_steps=40//64,epochs=30,
                          validation_data=test_transform)

Epoch 1/30
1/1 [==============================] - 3s 3s/step - loss: 0.6984 - acc: 0.5000
Epoch 2/30
1/1 [==============================] - 2s 2s/step - loss: 0.6450 - acc: 0.6579
Epoch 3/30
1/1 [==============================] - 2s 2s/step - loss: 0.7498 - acc: 0.4737
Epoch 4/30
1/1 [==============================] - 2s 2s/step - loss: 0.6022 - acc: 0.7656
Epoch 5/30
1/1 [==============================] - 2s 2s/step - loss: 0.6048 - acc: 0.6579
Epoch 6/30
1/1 [==============================] - 2s 2s/step - loss: 0.5812 - acc: 0.7344
Epoch 7/30
1/1 [==============================] - 2s 2s/step - loss: 0.4988 - acc: 0.8421
Epoch 8/30
1/1 [==============================] - 1s 1s/step - loss: 0.5729 - acc: 0.6842
Epoch 9/30
1/1 [==============================] - 2s 2s/step - loss: 0.5201 - acc: 0.7344
Epoch 10/30
1/1 [==============================] - 1s 1s/step - loss: 0.3355 - acc: 0.8684
Epoch 11/30
1/1 [==============================] - 1s 1s/step - loss: 0.5938 - acc: 0.7105
Epoch 12

In [23]:
model.save('cancer_ibm.h5')

In [24]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 24.3 MB/s eta 0:00:01


In [25]:
!tar -zcvf Lung_Cancer_Detection.tgz cancer_ibm.h5

cancer_ibm.h5


In [26]:
from ibm_watson_machine_learning import APIClient
wml_credentials={"url":"https://us-south.ml.cloud.ibm.com", "apikey": "ZmeYE5MslPa1WLpmvUfJ8jxIdwoiR2I-7s0rU-drMZdH"}
client=APIClient(wml_credentials)

In [27]:
def guide_from_space_name(client,space_name):
    space=client.spaces.get_details()
    return(next(item for item in space['resources']if item['entity']["name"]==space_name)['metadata']['id'])

In [37]:
space_uid=guide_from_space_name(client,'Lung')
print("Space UID =" + space_uid)

Space UID =f943d78e-21b8-4a7f-a6e7-38c63456f4c0


In [38]:
client.set.default_space(space_uid)

'SUCCESS'

In [39]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
scikit-learn_0.22-py3.6        154010fa-5b3b-4ac1-82af-4d5ee5abbc85  base
default_r3.6                   1b70aec3-ab34-4b87-8aa0-a4a3c8296a36  base
pytorch-onnx_1.3-py3.6         1bc6029

In [40]:
!pip install keras==2.2.4
import keras
keras.__version__

'2.2.4'

In [41]:
software_spec_uid=client.software_specifications.get_uid_by_name("tensorflow_2.4-py3.7")
software_spec_uid

'65e171d7-72d1-55d9-8ebb-f813d620c9bb'

In [33]:
!pip install watson-machine-learning-client

In [42]:
model_details=client.repository.store_model(model='Lung_Cancer_Detection.tgz',meta_props={client.repository.ModelMetaNames.NAME:"cancer_ibm",client.repository.ModelMetaNames.TYPE:"keras_2.2.4",client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid})
model_id=client.repository.get_model_uid(model_details)

Note: Warnings!! :  Model type keras_2.2.4 is deprecated. We recommend you use a supported model type. See Supported Frameworks https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/pm_service_supported_frameworks.html


In [43]:
model_id

'07ce1be2-e1cb-4d5e-8985-fad2d7764fc3'

In [44]:
client.repository.download(model_id,'Lung_CancerModel.tar.gz')

Successfully saved model content to file: 'Lung_CancerModel.tar.gz'


'/home/wsuser/work/Lung_CancerModel.tar.gz'